# 1. Load data and try to understand it

Data source: https://www.kaggle.com/gpreda/covid-world-vaccination-progress

Most likely something to do with unsupervised learning, but lets see.

In [ ]:
from datetime import datetime
import json

# Progress bar
from tqdm import tqdm

# Data
import pandas as pd
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

# Helper
from kneed import KneeLocator

# Algos
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#from sklearn.metrics import silhouette_score
#from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('data/country_vaccinations.csv')
df_man = pd.read_csv('data/country_vaccinations_by_manufacturer.csv')

## 1.2 What kind of data do we have here?

### 1.2.1 Country Vaccination

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().any()

In [ ]:
df.describe(include='all')

In [ ]:
', '.join(df.country.unique())

In [ ]:
', '.join(df.iso_code.unique())

In [ ]:
# Why do you have duplicated values? 
# Oh, nvm, a country can use multiple vaccines and they are comma-separated
', '.join(df.vaccines.unique()) 

In [ ]:
# Dec 2020 - May 2021
df.date.min(), df.date.max()

In [ ]:
df[df['iso_code'] == 'SWE'].sort_values(by='total_vaccinations', ascending=False).head()

### 1.2.2 Country Vaccination by Manufacturer

## 1.2 Features/Variables/Columns

Current features
- country
    - split into continents?
    - split the continents according geographical location? west, north, east, south, middle?
    - gdp?
- total_vaccinations, a bit of na values, fill?
- vaccines -> split mulitple into separate rows
- people_fully_vaccinated

Drop
- people_vaccinated
- daily_vaccinations_raw
- daily_vaccinations 	
- total_vaccinations_per_hundred 	
- people_vaccinated_per_hundred 	
- people_fully_vaccinated_per_hundred 	
- daily_vaccinations_per_million 	
- source_name 	
- source_website
- iso_code

New features
- [inhabitants/population](https://www.kaggle.com/tanuprabhu/population-by-country-2020)

In [ ]:
df.info()

# 2 Fix fix the data

In [ ]:
drop_columns = [
    "people_vaccinated",
    "people_fully_vaccinated",
    "daily_vaccinations_raw",
    "daily_vaccinations",
    "total_vaccinations_per_hundred",
    "people_vaccinated_per_hundred",
    "people_fully_vaccinated_per_hundred",
    "daily_vaccinations_per_million",
    "source_name",
    "source_website",
    "iso_code",
]

df = df.drop(drop_columns, axis=1)
df.info()

## 2.1 Fill out NA

In [ ]:
na_country_set = set()
    
for col in ['total_vaccinations']:
    for i in df[df[col].isna()].index:
        na_country_set.add(df['country'][i])

    for c in na_country_set:
        last = None
        stack = []
        for i in df[df['country'] == c].index:
            date = df['date'][i]
            curr = df[col][i]
            if np.isnan(curr):
                stack.append(i)
                continue

            if len(stack) == 0 and not np.isnan(curr):
                last = curr
                continue

            # Found end
            try:
                increments = (curr - last) / len(stack)
            except TypeError as e:
                continue

            while stack:
                stack_len = len(stack)
                curr_idx = stack.pop()
                df.at[curr_idx, col] = np.float64(int(last + increments*stack_len))

            last = curr
        
    df[col].fillna(method='ffill', inplace=True)
    
    
    for i in df[df['country'] == 'United States'].index:
        date = df['date'][i]
        curr = df[col][i]
        print(
            'USA {} {}'.format(
                date, curr
            )
        )
        
    na_country_set = set()

In [ ]:
df.info()

## 2.2 Group countries into groups

In [ ]:
# Sort countries by their mean vaccines
GROUP_N = 7
tvs = df[['country', 'total_vaccinations']].groupby('country').mean().sort_values(by='total_vaccinations', ascending=False)
bucket = dict()
count = 1
inner_count = 0
for c in tvs.index:
    if inner_count == GROUP_N:
        count += 1
        inner_count = 0
        
    bucket[c] = count
    inner_count += 1
    

# Introduce new column to graph, total_vaccines_grouped into GROUP_N
df['country_group'] = df['country'].map(bucket)

In [ ]:
plt.figure(figsize=(16, 16))
sns.lineplot(data=df[df['country_group'] == 1], x="date", y="total_vaccinations", hue='country')

Why is 2020 not on the left side? Just covert it into epoch...

In [ ]:
# To unix time
for i in df.index:
    d = df['date'][i].split('-')
    df.at[i, 'date_unix'] = datetime(int(d[0]), int(d[1]), int(d[2])).timestamp()
    
plt.figure(figsize=(16, 10))
sns.lineplot(data=df[df['country_group'] == 1], x="date_unix", y="total_vaccinations", hue='country')
plt.legend(bbox_to_anchor=(1.1, 1), borderaxespad=0)

In [ ]:
#g = sns.FacetGrid(df.query('country_group == 1 or country_group == 2'), col='country_group', col_wrap=2, sharey=False, height=10)
g = sns.FacetGrid(df, col='country_group', col_wrap=2, sharey=False, height=10)
                  
g.map(sns.lineplot, 'date_unix', 'total_vaccinations', 'country')
for ax in g.axes.ravel():
    ax.legend(loc='upper left')

## 2.3 total_vaccinations per person ratio

In [ ]:
df_pop = pd.read_csv('data/population_by_country_2020.csv')
df_pop.rename(columns={
    'Country (or dependency)': 'country',
    'Population (2020)': 'population',
}, inplace=True)
df_pop.info()

### 2.3.1 Check countries match with original dataset and fill missing

In [ ]:
countries = df.country.unique()
not_found_countries = list()

for c in countries:
    pop = df_pop.loc[df_pop['country'].str.contains(c), 'population'].values
    if len(pop) == 0:
        print('could not find population for {} in dataset'.format(c))
        not_found_countries.append(c)
        
replace = {
    'Cabo Verde': 'Cape Verde',
    "Côte d'Ivoire": "Cote d'Ivoire",
    'DR Congo': 'Democratic Republic of Congo',
    'Saint Kitts & Nevis': 'Saint Kitts and Nevis',
    'St. Vincent & Grenadines': 'Saint Vincent and the Grenadines',
    'Sao Tome & Principe': 'Sao Tome and Principe',
    'Turks and Caicos': 'Turks and Caicos Islands',
}

add = {
    'Curacao': 157538, # 2019, Världsbanken
    'England': 55.98 * (10**6), # 2018, ONS Storbritannien, Eurostat, Världsbanken
    'Guernsey': 62792, # 2019, wiki? google?
    'Jersey': 97857, # 2011, wiki? google?
    'Kosovo': 1.873 * (10**6), # 2020, wiki? google?
    'Northern Cyprus': 326000, # 2017, wiki? google?
    'Northern Ireland': 1.885 * (10**6), # 2019, Eurostat, Förenta nationerna
    'Scotland': 5.454 * (10**6), # 2019, Eurostat
    'Wales': 3.136 * (10**6), # 2019, Eurostat
}

# Replace
for old, new in replace.items():
    df_pop.at[df_pop[df_pop['country'] == old].index[0], 'country'] = new
    
# Add
for name, pop in add.items():
    df_pop = df_pop.append({
        'country': name,
        'population': pop,
    }, ignore_index=True)

In [ ]:
for name in add:
    print(df_pop[df_pop['country'] == name][['country', 'population']])

### 2.3.2 Ratio feature

`total_vaccination / population`

`if > 1`: most likely the entire population got the first vaccine (assuming you randomly vaccinate people)

In [ ]:
for i in df.index:
    tvc = df['total_vaccinations'][i]
    country = df['country'][i]
    pop = df_pop[df_pop['country'].str.contains(country)].population.values[0]
    df.at[i, 'total_vaccinations_population_ratio'] = tvc/pop

In [ ]:
g = sns.FacetGrid(df, col='country_group', col_wrap=2, sharey=False, height=10)
g.map(sns.scatterplot, 'date_unix', 'total_vaccinations_population_ratio', 'country')
for ax in g.axes.ravel():
    ax.legend(loc='upper left')

In [ ]:
plt.figure(figsize=(16,32))
sns.barplot(data=df.sort_values(by='country'), x='total_vaccinations_population_ratio', y='country')

## 2.4 Split out vaccines

In [ ]:
for v in df.vaccines.unique():
    print(v)

In [ ]:
SPLIT_VACCINES_CSV_PATH = 'data/custom_split_vaccines.csv'
df_splitv = pd.DataFrame()
try:
    # 'Cached'
    df_splitv = pd.read_csv(SPLIT_VACCINES_CSV_PATH)
    df = pd.read_csv('data/custom_country_vaccinations.csv')
except Exception as e:
    # CSV does most likely to exist
    df_splitv = pd.DataFrame(columns=np.append(df.columns.values, 'vaccine'))
    idx_to_delete = list()

    for v in tqdm(df.vaccines.unique()):
        spl = v.split(', ')
        cols = df[df['vaccines'] == v]
        for i in cols.index:
            if len(spl) == 1:
                df.at[i, 'vaccine'] = spl[0]
                continue
                
            for vs in spl:
                curr = df.iloc[i].copy()
                curr['vaccine'] = vs
                df_splitv = df_splitv.append(curr, ignore_index=True)

        idx_to_delete.extend(cols.index)
    
    df_splitv.to_csv('data/custom_split_vaccines.csv', index=False)
    df = df.drop(index=idx_to_delete)
    df = df.append(df_splitv, ignore_index=True)
    
#df = df.drop(columns=['vaccines'], axis=1)
#df.to_csv('data/custom_country_vaccinations.csv', index=False)
df = df.astype({'country_group': int})
df.info()

In [ ]:
df[['vaccine', 'country']].groupby('vaccine').nunique().sort_values(by='country', ascending=False)

In [ ]:
v_map = dict()

uniqv = df[['vaccine', 'country']].groupby('vaccine').nunique().sort_values(by='country', ascending=False).to_dict()['country']
count = 1
for k in uniqv:
    v_map[k] = count
    count += 1
    
df = df.replace({'vaccine': v_map})
print(json.dumps(v_map, indent=2))

In [ ]:
df.sort_values(by='total_vaccinations', ascending=False).head()

# 3 Algos 

In [ ]:
print(df.columns.values)
test = df.copy()
test.info()

## 3.1 K-Means

https://realpython.com/k-means-clustering-python/#understanding-the-k-means-algorithm

https://www.youtube.com/watch?v=4b5d3muPQmA

In [ ]:
#kmeans_drop_columns = ['country', 'date'] # n=2 elbow
kmeans_drop_columns = ['country', 'country_group', 'date', 'date_unix', 'total_vaccinations', 'vaccine'] # n=3 elbow
kmeans_kwargs = {
    'init': 'random',
    'n_init': 10,
    'max_iter': 300,
    'random_state': 42
}

ran = range(1, 11)

### 3.1.1 Elbow method

https://en.wikipedia.org/wiki/Residual_sum_of_squares

In [ ]:
sse = []
for k in ran:
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(test.drop(columns=kmeans_drop_columns, axis=1))
    sse.append(kmeans.inertia_)
    
clusters_n = KneeLocator(ran, sse, curve='convex', direction='decreasing').elbow
print('KneeLocator', clusters_n)
sns.lineplot(x=ran, y=sse)

### 3.1.2 Silhouette method

In [ ]:
# Takes time...

if False:
    silhouette_coefficients = list()
    ran = range(2, 11)

    for k in tqdm(ran):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(test.drop(columns=['country', 'date'], axis=1))
        score = silhouette_score(test.drop(columns=['country', 'date'], axis=1), kmeans.labels_)
        silhouette_coefficients.append(score)

    sns.lineplot(x=ran, y=silhouette_coefficients)

### 3.1.3 Where do the clusters form?

`total_vaccinations` above a certain threshold becomes a cluster... not that insightful?

In [ ]:
kmeans = KMeans(n_clusters=clusters_n)
test['clusters'] = kmeans.fit_predict(test.drop(columns=kmeans_drop_columns, axis=1))
print(test.head())

cols = test.drop(columns=['country', 'date']).columns.values

for x in cols:
    for y in cols:
        if x == y:
            continue
        
        if x == 'clusters' or y == 'clusters':
            continue
            
        plt.figure(figsize=(10, 10))
        sns.scatterplot(data=test, x=x, y=y, hue='clusters')
        plt.show()

Clustered based on date_unix?

In [ ]:
plt.figure(figsize=(16,32))
g = sns.scatterplot(data=test, x="date_unix", y="country_group", size="total_vaccinations", legend=False, hue='clusters', sizes=(20, 2000))
plt.show()

In [ ]:
plt.figure(figsize=(16,32))
sns.barplot(data=test.sort_values(by='country'), x="total_vaccinations", y="country", hue='clusters')
plt.show()

In [ ]:
test[['total_vaccinations', 'clusters', 'date_unix']].groupby('clusters').agg(['mean', 'min', 'max'])


In [ ]:
for n in range(0, clusters_n):
    print(f'-----Cluster {n}-----')
    meanv = test[test['clusters'] == n]['total_vaccinations'].mean()
    minv = test[test['clusters'] == n]['total_vaccinations'].min()
    maxv = test[test['clusters'] == n]['total_vaccinations'].max()
    print(f'mean vaccinations:\t{round(meanv, 2)}')
    print(f'min max vaccinations:\t{minv} <= {maxv}')
    
    meanv = test[test['clusters'] == n]['total_vaccinations_population_ratio'].mean()
    minv = test[test['clusters'] == n]['total_vaccinations_population_ratio'].min()
    maxv = test[test['clusters'] == n]['total_vaccinations_population_ratio'].max()
    print(f'vacc/pop min max:\t{round(minv, 3)} <= {round(maxv, 3)}')
    print(f'vacc/pop mean:\t\t{round(meanv, 3)}')
    
    minv = test[test['clusters'] == n]['date_unix'].min()
    maxv = test[test['clusters'] == n]['date_unix'].max()
    print(f'min max date:\t\t{datetime.fromtimestamp(minv)} <= {datetime.fromtimestamp(maxv)}')
    
    inv_v_map = {v: k for k, v in v_map.items()}
    unique_vaccines = test[test['clusters'] == n].vaccine.unique()
    unique_vaccines_str = list()
    for i in range(len(unique_vaccines)):
        name = inv_v_map[unique_vaccines[i]]
        unique_vaccines_str.append(name)
        
    print(f'unique vaccines:\t{", ".join(unique_vaccines_str)}, ({len(unique_vaccines)})')
    
    unique_countries = test[test['clusters'] == n].country.unique()
    print(f'unique countries:\t{", ".join(unique_countries)}, ({len(unique_countries)})')
    
    
    
    print()


In [ ]:
g = sns.FacetGrid(test, col='clusters', height=5, sharey=False, sharex=False, col_wrap=1)
g.map(sns.scatterplot, 'date_unix', 'total_vaccinations', 'country')
for ax in g.axes.ravel():
    ax.legend(bbox_to_anchor=(2.05, 1))

In [ ]:
g = sns.FacetGrid(test, col='clusters', height=5, aspect=1.5, sharey=False, sharex=False, col_wrap=1)
g.map(sns.scatterplot, 'date_unix', 'total_vaccinations_population_ratio', 'country')
for ax in g.axes.ravel():
    ax.legend(bbox_to_anchor=(1.5, 1))